# Flatiron School Data Science Project 4

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [2]:
spark = SparkSession.builder.appName("Movie Recommender").getOrCreate()

25/03/21 10:08:58 WARN Utils: Your hostname, Matthews-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.129.2.122 instead (on interface en0)
25/03/21 10:08:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/21 10:08:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
ratings_df = spark.read.csv("./data/ratings.csv", header=True, inferSchema=True)
ratings_df = ratings_df.repartition(500, "userId")

In [8]:
(training, test) = ratings_df.randomSplit([.8, .2])
small_train = training.sample(.05)
small_test = test.sample(.1)

In [9]:
als = ALS(
    maxIter=5, regParam=.1, userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop", numUserBlocks=100, numItemBlocks=100
)

model = als.fit(small_train)

25/03/21 10:17:23 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
25/03/21 10:17:28 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [16]:
model.getParam("predictionCol")

Param(parent='ALS_6b51b74a7592', name='predictionCol', doc='prediction column name.')

In [10]:
predictions = model.transform(small_test)

In [18]:
predictions.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)
 |-- prediction: float (nullable = false)



In [11]:
evaluator = RegressionEvaluator(
    metricName="mae", 
    labelCol="rating",
    predictionCol="prediction"
)
mae = evaluator.evaluate(predictions)
print(f"Mean Absolute Error (MAE) = {mae}")

Mean Absolute Error (MAE) = 0.8707650261906273
